Завдання 1: Завантажити для кожної адміністративної одиниці України тестові файли з VHI-індексом за допомогою urllib.

In [1]:
#Визначення функції
import os
import pandas as pd
import urllib.request
import datetime
import glob

if not os.path.exists("data"):
    os.makedirs("data")

def download_vhi(province_id):
    """
    Завантаження VHI даних для області
    """
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1=1981&year2=2024&type=Mean"
    
    existing_files = glob.glob(f"data/VHI_{province_id}_*.csv")
    if existing_files:
        print(f"Область {province_id} вже завантажена.")
        return
    
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"data/VHI_{province_id}_{timestamp}.csv"
    
    urllib.request.urlretrieve(url, filename)
    print(f"Файл збережено: {filename}")

In [2]:
#Виклик
for province_id in range(1, 27):
    download_vhi(province_id)

Область 1 вже завантажена.
Область 2 вже завантажена.
Область 3 вже завантажена.
Область 4 вже завантажена.
Область 5 вже завантажена.
Область 6 вже завантажена.
Область 7 вже завантажена.
Область 8 вже завантажена.
Область 9 вже завантажена.
Область 10 вже завантажена.
Область 11 вже завантажена.
Область 12 вже завантажена.
Область 13 вже завантажена.
Область 14 вже завантажена.
Область 15 вже завантажена.
Область 16 вже завантажена.
Область 17 вже завантажена.
Область 18 вже завантажена.
Область 19 вже завантажена.
Область 20 вже завантажена.
Область 21 вже завантажена.
Область 22 вже завантажена.
Область 23 вже завантажена.
Область 24 вже завантажена.
Область 25 вже завантажена.
Область 26 вже завантажена.


Завдання 2: Зчитати завантажені файли у pandas DataFrame. Виконати data cleaning

In [3]:
#визначення ф-ції
def load_and_clean():
    files = glob.glob("data/*.csv")
    all_data = []

    for file in files:
        province_id = int(file.split("_")[1])

        # читаємо файл як текст
        with open(file, "r", encoding="latin1") as f:
            lines = f.readlines()

        # залишаємо тільки ті рядки, що починаються з цифри (тобто дані)
        data_lines = [line for line in lines if line.strip() and line.strip()[0].isdigit()]

        # створюємо DataFrame вручну
        rows = []
        for line in data_lines:
            parts = line.strip().split(",")
            if len(parts) >= 7:
                rows.append(parts[:7])

        df = pd.DataFrame(rows, columns=["year", "week", "smn", "smt", "vci", "tci", "vhi"])

        # перетворюємо в числа
        df = df.apply(pd.to_numeric, errors="coerce")
        df = df.dropna()

        df["ProvinceID"] = province_id
        all_data.append(df)

    return pd.concat(all_data, ignore_index=True)

In [4]:
#Виклик
df_vhi = load_and_clean()
df_vhi.head()

,year,week,smn,smt,vci,tci,vhi,ProvinceID
0,1982,2,0.063,261.53,55.89,38.20,47.04,10
1,1982,3,0.063,263.45,57.30,32.69,44.99,10
2,1982,4,0.061,265.10,53.96,28.62,41.29,10
3,1982,5,0.058,266.42,46.87,28.57,37.72,10
4,1982,6,0.056,267.47,39.55,30.27,34.91,10


Завдання 3: Змінити індексацію областей з англійської (NOAA) на українську абетку.

In [5]:
#Визначення ф-ції
ukrainian_alphabet_index = {
    1: "Вінницька", 2: "Волинська", 3: "Дніпропетровська",
    4: "Донецька", 5: "Житомирська", 6: "Закарпатська",
    7: "Запорізька", 8: "Івано-Франківська", 9: "Київська",
    10: "Кіровоградська", 11: "Луганська", 12: "Львівська",
    13: "Миколаївська", 14: "Одеська", 15: "Полтавська",
    16: "Рівненська", 17: "Сумська", 18: "Тернопільська",
    19: "Харківська", 20: "Херсонська", 21: "Хмельницька",
    22: "Черкаська", 23: "Чернівецька", 24: "Чернігівська",
    25: "АР Крим", 26: "м. Київ"
}

noaa_to_ua = {
    1:22,2:24,3:23,4:25,5:3,6:4,7:8,8:19,9:20,
    10:21,11:9,12:26,13:10,14:11,15:12,
    16:13,17:14,18:15,19:16,20:17,
    21:18,22:6,23:1,24:2,25:7,26:5
}

def reindex_provinces(df):
    df["UA_Index"] = df["ProvinceID"].map(noaa_to_ua)
    df["ProvinceName"] = df["UA_Index"].map(ukrainian_alphabet_index)
    return df

In [6]:
#Виклик
df_vhi = reindex_provinces(df_vhi)
df_vhi.head()

,year,week,smn,smt,vci,tci,vhi,ProvinceID,UA_Index,ProvinceName
0,1982,2,0.063,261.53,55.89,38.20,47.04,10,21,Хмельницька
1,1982,3,0.063,263.45,57.30,32.69,44.99,10,21,Хмельницька
2,1982,4,0.061,265.10,53.96,28.62,41.29,10,21,Хмельницька
3,1982,5,0.058,266.42,46.87,28.57,37.72,10,21,Хмельницька
4,1982,6,0.056,267.47,39.55,30.27,34.91,10,21,Хмельницька


Завдання 4.1: Отримати ряд VHI для області за вказаний рік.

In [7]:
#Визначення ф-ції
def get_vhi_by_year(df, province_name, year):
    result = df[(df["ProvinceName"] == province_name) & (df["year"] == year)]
    return result[["week", "vhi"]]

In [8]:
#Виклик
get_vhi_by_year(df_vhi, "Київська", 2020)

,week,vhi
4210,1,37.78
4211,2,38.41
4212,3,39.74
4213,4,41.90
4214,5,43.53
4215,6,43.37
4216,7,41.50
4217,8,39.53
4218,9,38.90
4219,10,39.07


Завдання 4.2: Отримати VHI за діапазон років для вказаних областей.

In [9]:
#визначення ф-ції
def get_vhi_range(df, provinces, start_year, end_year):
    result = df[
        (df["ProvinceName"].isin(provinces)) &
        (df["year"] >= start_year) &
        (df["year"] <= end_year)
    ]
    return result[["ProvinceName", "year", "week", "vhi"]]

In [10]:
#Виклик
get_vhi_range(df_vhi, ["Київська", "Львівська"], 2000, 2010)

,ProvinceName,year,week,vhi
3170,Київська,2000,1,25.03
3171,Київська,2000,2,26.69
3172,Київська,2000,3,28.26
3173,Київська,2000,4,30.80
3174,Київська,2000,5,33.90
...,...,...,...,...
12677,Львівська,2010,48,42.77
12678,Львівська,2010,49,40.63
12679,Львівська,2010,50,38.26
12680,Львівська,2010,51,36.91


Завдання 4.3: Знайти мінімум, максимум, середнє та медіану VHI.

In [11]:
#Визначення ф-ції
def get_statistics(df, provinces, start_year, end_year):
    subset = df[
        (df["ProvinceName"].isin(provinces)) &
        (df["year"] >= start_year) &
        (df["year"] <= end_year)
    ]
    
    return {
        "min": subset["vhi"].min(),
        "max": subset["vhi"].max(),
        "mean": subset["vhi"].mean(),
        "median": subset["vhi"].median()
    }

In [12]:
#Виклик
get_statistics(df_vhi, ["Київська", "Львівська"], 2000, 2020)

{'min': np.float64(-1.0),
 'max': np.float64(80.88),
 'mean': np.float64(47.91442307692308),
 'median': np.float64(48.015)}